# EGP sans BatchDC

Kerry N. Wood

kerry.wood@asterism.ai

January 21, 2025

We often want a type-0 TLE from a type-4 one.  The AstroStandards has routines for this, but they're hidden in Batch (and thus ITAR).  Write an open-source version.

Do NOT hide any functionality in nice routines.  Show all the sausage.

In [1]:
import time
from datetime import datetime, timedelta
import numpy as np
import scipy.optimize

# load the AstroStandards harnesses
from load_utils import *        # load utils...
import helpers                  # helpers that come with our harness

****************************************************************************************************
Remember when converting dates/times that you should use Julian dates to avoid leap second issues
   helpers.DS50EPOCH + <days_since_50_float> * u.day ---> WRONG
   astropy.time.Time( helpers.DS50EPOCH.jd + <days_since_50_float> , format='jd') ---> CORRECT
****************************************************************************************************


## Input data

In [2]:
# example TLE 
# this is a type-4 faked by a modified from a space-track TLE
L1 = '1 25544U 98067A   24365.67842578  .00026430  00000-0  46140-3 4  9990'
L2 = '2 25544  51.6404  61.8250 0005853  25.4579 117.0387 15.50482079489028'

# what dates should we convert to?
WINDOW_START = datetime( year=2025, month=1, day=10 )
WINDOW_END   = datetime( year=2025, month=1, day=11 )

In [3]:
# init the astrostandards and get a version string
init_all()
# check the version of what you loaded
infostr = Cstr('',128)
DllMainDll.DllMainGetInfo( infostr )
print('I see version as :\n\t{}'.format( infostr.value.decode('utf-8') ) )

EnvConstDll: 0
TimeFuncDll: 0
AstroFuncDll: 0
TleDll: 0
SpVecDll: 0
VcmDll: 0
ExtEphemDll: 0
Sgp4PropDll: 0
SpPropDll: 0
ElOpsDll: 0
SatStateDll: 0
SensorDll: 0
ObsDll: 0
ObsOpsDll: 0
LamodDll: 0
RotasDll: 0
BatchDCDll: 0
b''
I see version as :
	HQ SpOC DllMain - Version: v9.4 - Build: May 03 2024 - Platform: Linux 64-bit - Compiler: OneAPI ifort                          


#### Generate truth data

In [4]:
# clear all sats
TleDll.TleRemoveAllSats()
# load the TLE
tleid = TleDll.TleAddSatFrLines( Cstr(L1,512), Cstr(L2,512) )

# ----------------------------------------------------------------------------------------------
# now, pull the data back out of AstroStandards and into an "introspector" helper class
# per the code, these are the XA_TLE lookups... init a helper to parse those data
XA_TLE = helpers.astrostd_named_fields( TleDll, prefix='XA_TLE_') 
# it also passes back some string values.. we'll ignore those
XS_TLE = Cstr('',512)

# now pull..
TleDll.TleDataToArray( tleid, XA_TLE.data, XS_TLE )  # <--- note that you pass the "data" holder in
origvals = XA_TLE.toDict()
# print the data
print(origvals)

{'XA_TLE_SATNUM': 25544.0, 'XA_TLE_EPOCH': 27393.67842578, 'XA_TLE_NDOT': 0.0002643, 'XA_TLE_SP_BTERM': 0.0002643, 'XA_TLE_NDOTDOT': 0.0, 'XA_TLE_SP_OGPARM': 0.0, 'XA_TLE_BSTAR': 0.0, 'XA_TLE_SP_AGOM': 0.0, 'XA_TLE_EPHTYPE': 4.0, 'XA_TLE_INCLI': 51.6404, 'XA_TLE_NODE': 61.825, 'XA_TLE_ECCEN': 0.0005853, 'XA_TLE_OMEGA': 25.4579, 'XA_TLE_MNANOM': 117.0387, 'XA_TLE_MNMOTN': 15.50482079, 'XA_TLE_REVNUM': 48902.0, 'XA_TLE_ELSETNUM': 999.0, 'XA_TLE_ORGSATNUM': 0.0, 'XA_TLE_BTERM': 0.0004614, 'XA_TLE_OBSTIME': 0.0, 'XA_TLE_EGR': 0.0, 'XA_TLE_EDR': 0.0, 'XA_TLE_VISMAG': 0.0, 'XA_TLE_RCS': 0.0, 'XA_TLE_AGOMGP': 0.0}


## Fitting

So, how will we do this?

- copy over the data from the original TLE (as our seed orbit), but change the type to 0
- specify which fields we will optimize over
- let Python twiddle the bits until we have an answer that best matches our output ephemeris

In [5]:
# truth ephemeris; pick one minute spacing because reasons
# convert dates to the astrostandards formats using our helpers
WINDOW_START_DS50 = helpers.datetime_to_ds50( WINDOW_START, TimeFuncDll )
WINDOW_END_DS50   = helpers.datetime_to_ds50( WINDOW_END, TimeFuncDll )
DS50_DATES        = np.arange( WINDOW_START_DS50, WINDOW_END_DS50, 1/1440 )  # astrostandards does days since 1950.. this is one minute steps

In [6]:
# helper routine to turn data arrays into TLE lines (using astrostandards)
def arrayToTle( HELPER : helpers.astrostd_named_fields ):
    TleDll.TleRemoveAllSats()
    tleid = TleDll.TleAddSatFrArray( HELPER.data, XS_TLE )
    assert tleid > 0
    outL1, outL2 = Cstr('',512), Cstr('',512)
    assert TleDll.TleGetLines( tleid, outL1, outL2 ) == 0
    return outL1.value.decode('utf-8'), outL2.value.decode('utf-8')

In [7]:
# -------------------------------------------------------------------------------------
# propagate routine...
# start with our C-handles to the variables
mse = ctypes.c_double()
pos = (ctypes.c_double * 3)()
vel = (ctypes.c_double * 3)()
llh = (ctypes.c_double * 3)()
def propTle( tleid, ds50 : list ):
    ''' propagate initialized tle to a bunch of dates, return matrix'''
    def propDS50( tleid, date ):
        Sgp4PropDll.Sgp4PropDs50UTC( tleid, date, mse, pos, vel, llh )
        return np.hstack( (date, float(mse.value), list(pos), list(vel)) )   # < -- note the use of list / float to get copies
    return np.vstack( [ propDS50(tleid,D) for D in ds50 ] )

# init the Sgp4 propagator on the tle
assert 0 == Sgp4PropDll.Sgp4InitSat( tleid )
# now propagate
truth  = propTle( tleid, DS50_DATES )
# truth

In [8]:
# get another data holder and populate it with the same data as the seed
TEST_TLE = helpers.astrostd_named_fields( TleDll, prefix='XA_TLE_') 
# use a conversion routine to convert to the array *without* loading
TleDll.TleLinesToArray( Cstr(L1,512), Cstr(L2,512), TEST_TLE.data, XS_TLE )
# switch the type from 4 to zero... 
TEST_TLE['XA_TLE_EPHTYPE'] = 0.
# set the epoch to the start of our interval
TEST_TLE['XA_TLE_EPOCH'] = (WINDOW_START_DS50 + WINDOW_END_DS50) / 2

In [9]:
TEST_TLE.toDict()

{'XA_TLE_SATNUM': 25544.0,
 'XA_TLE_EPOCH': 27404.5,
 'XA_TLE_NDOT': 0.0002643,
 'XA_TLE_SP_BTERM': 0.0002643,
 'XA_TLE_NDOTDOT': 0.0,
 'XA_TLE_SP_OGPARM': 0.0,
 'XA_TLE_BSTAR': 0.0,
 'XA_TLE_SP_AGOM': 0.0,
 'XA_TLE_EPHTYPE': 0.0,
 'XA_TLE_INCLI': 51.6404,
 'XA_TLE_NODE': 61.825,
 'XA_TLE_ECCEN': 0.0005853,
 'XA_TLE_OMEGA': 25.4579,
 'XA_TLE_MNANOM': 117.0387,
 'XA_TLE_MNMOTN': 15.50482079,
 'XA_TLE_REVNUM': 48902.0,
 'XA_TLE_ELSETNUM': 999.0,
 'XA_TLE_ORGSATNUM': 0.0,
 'XA_TLE_BTERM': 0.0004614,
 'XA_TLE_OBSTIME': 0.0,
 'XA_TLE_EGR': 0.0,
 'XA_TLE_EDR': 0.0,
 'XA_TLE_VISMAG': 0.0,
 'XA_TLE_RCS': 0.0,
 'XA_TLE_AGOMGP': 0.0}

In [10]:
# reset the seed state to the new epoch, first find the closest date to the epoch
new_epoch = TEST_TLE['XA_TLE_EPOCH']
idx       = np.argmin( np.abs( truth[:,0] - new_epoch ) )
new_sv    = truth[ idx ]
P,V       = truth[idx,2:5], truth[idx,5:]

# we'll use the conversion in the astrostandards
XA_KEP    = helpers.astrostd_named_fields( AstroFuncDll,  prefix='XA_KEP_' )
AstroFuncDll.PosVelToKep( (ctypes.c_double*3)(*P), (ctypes.c_double*3)(*V), XA_KEP.data )

# now set the values
TEST_TLE['XA_TLE_INCLI']  = XA_KEP['XA_KEP_INCLI']
TEST_TLE['XA_TLE_NODE']   = XA_KEP['XA_KEP_NODE']
TEST_TLE['XA_TLE_ECCEN']  = XA_KEP['XA_KEP_E']
TEST_TLE['XA_TLE_MNANOM'] = XA_KEP['XA_KEP_MA']
TEST_TLE['XA_TLE_OMEGA']  = XA_KEP['XA_KEP_OMEGA']
TEST_TLE['XA_TLE_OMEGA']  = XA_KEP['XA_KEP_OMEGA']
TEST_TLE['XA_TLE_MNMOTN'] = AstroFuncDll.AToN( XA_KEP['XA_KEP_A'] )

# save those as our seedvals
seedvals = TEST_TLE.toDict()

In [43]:
# what fields will we optimize over?  This doubles as a field accessor list for the optimizer..
FIELDS = [
'XA_TLE_BTERM',
'XA_TLE_NDOT',
'XA_TLE_SP_BTERM',
'XA_TLE_INCLI',
'XA_TLE_NODE',
'XA_TLE_ECCEN',
'XA_TLE_OMEGA',
'XA_TLE_MNANOM',
'XA_TLE_MNMOTN',
]

def optFunction( X, FIELDS, seedvals, return_scalar=True ):
    # --------------------- set our seed values (all vals)
    for k,v in seedvals.items(): TEST_TLE[ k ] = v
    # --------------------- now use modified values
    for k,v in zip(FIELDS,X) :   TEST_TLE[ k ] = v

    # --------------------- clear state
    TleDll.TleRemoveAllSats()
    Sgp4PropDll.Sgp4RemoveAllSats()

    # --------------------- init our test TLE from the modified data
    tleid = TleDll.TleAddSatFrArray( TEST_TLE.data, XS_TLE )
    if tleid <= 0: return np.inf
    if Sgp4PropDll.Sgp4InitSat( tleid ) != 0: return np.inf
    
    # --------------------- generate our test ephemeris
    test_tle = propTle( tleid, DS50_DATES )
    
    # use numpy to return the distance between our hypothesis and truth
    resids = test_tle[:,2:] - truth[:,2:]
    rms    = np.sqrt( np.sum( np.linalg.norm( resids, axis=1 ) ) / resids.shape[0] )
    print( rms , end='\r')
    if return_scalar:
        return rms
        # return np.sum( np.linalg.norm( resids[:,:3], axis=1 ) ) 
    else:
        np.linalg.norm( resids[:,:3], axis=1 ) 

In [44]:
st= time.time()

# get an initial simplex that is rich in entropy
# seed  = np.array( [ seedvals[k] for k in FIELDS ] )
# N     = len(seed)
# smplx = np.random.uniform(0,.5,size=(N+1,N))
# smplx += seed 

# -----------------------------  nelder mead -----------------------------
# if your seed is not near the final, nelder works great (at the expense of time)
ans = scipy.optimize.minimize( optFunction, 
                               [ seedvals[k] for k in FIELDS ],
                               args    = (FIELDS,seedvals,True),
                               method  = 'Nelder-Mead' )
                               # options = {'initial_simplex' : smplx } )

# -----------------------------  least_sq  -----------------------------
# if your seed IS near the final, follow the gradient
# ans = scipy.optimize.least_squares( optFunction, 
#                                [ seedvals[k] for k in FIELDS ],
#                                args = (FIELDS,seedvals, False) )

print('Optimization / conversion took {:8.3f} seconds'.format( time.time() - st ) )

Optimization / conversion took   19.696 seconds


In [37]:
print('Your original TLE was')
print('\n'.join( [L1,L2] ) ) 

# what was our seed value
for k,v in seedvals.items(): TEST_TLE[ k ] = v
print('Your seed TLE was')
print('\n'.join(arrayToTle( TEST_TLE )))

# now update with perturbed values
for k,v in zip(FIELDS,ans.x) :   TEST_TLE[ k ] = v
print('Your updated TLE is')
print('\n'.join(arrayToTle( TEST_TLE )))

Your original TLE was
1 25544U 98067A   24365.67842578  .00026430  00000-0  46140-3 4  9990
2 25544  51.6404  61.8250 0005853  25.4579 117.0387 15.50482079489028
Your seed TLE was
1 25544U 98067A   25010.50000000 +.00026430  00000 0  00000 0 0 0999                                                                                                                                                                                                                                                                                                                                                                                                                                                            
2 25544  51.6266   8.1144 0013106  21.2258  91.5451 15.5193427848902                                                                                                                                                                                                                                               

In [38]:
for i,F in enumerate(FIELDS):
    print('{:15} {:12.8f} {:12.8f}'.format(F, seedvals[F], ans.x[i] ) )

XA_TLE_BTERM      0.00046140   0.00207021
XA_TLE_NDOT       0.00026430   0.00009262
XA_TLE_SP_BTERM   0.00026430  -0.00002658
XA_TLE_INCLI     51.62663150  51.64055297
XA_TLE_NODE       8.11444312   8.13285676
XA_TLE_ECCEN      0.00131061   0.00087751
XA_TLE_OMEGA     21.22582332  -6.60393936
XA_TLE_MNANOM    91.54514763 119.39073180
XA_TLE_MNMOTN    15.51934278  15.50676752


In [39]:
ans

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 49.34369082563087
             x: [ 2.070e-03  9.262e-05 -2.658e-05  5.164e+01  8.133e+00
                  8.775e-04 -6.604e+00  1.194e+02  1.551e+01]
           nit: 753
          nfev: 1167
 final_simplex: (array([[ 2.070e-03,  9.262e-05, ...,  1.194e+02,
                         1.551e+01],
                       [ 2.070e-03,  9.262e-05, ...,  1.194e+02,
                         1.551e+01],
                       ...,
                       [ 2.070e-03,  9.262e-05, ...,  1.194e+02,
                         1.551e+01],
                       [ 2.070e-03,  9.262e-05, ...,  1.194e+02,
                         1.551e+01]]), array([ 4.934e+01,  4.934e+01,  4.934e+01,  4.934e+01,
                        4.934e+01,  4.934e+01,  4.934e+01,  4.934e+01,
                        4.934e+01,  4.934e+01]))